In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math


In [3]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)

        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]


In [4]:
class DiffMultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(DiffMultiHeadAttention, self).__init__()

        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)

        self.dense = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)

    def diff_attention(self, Q, K, V):
        d_k = Q.size(-1)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(d_k)
        A1 = F.softmax(scores, dim=-1)
        A2 = F.softmax(scores, dim=-1)
        diff = A1 - A2
        
        return torch.matmul(diff, V)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        output = self.diff_attention(q, k, v)
        
        output = output.permute(0, 2, 1, 3).contiguous()
        output = output.view(batch_size, -1, self.d_model)
        
        return self.dense(output)


In [5]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(FeedForward, self).__init__()
        
        self.linear_a = nn.Linear(d_model, d_ff)
        self.linear_b = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.linear_b(nn.ReLU()(self.linear_a(x)))


In [6]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        
        self.diff_multi_head_attention = DiffMultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)
        
        self.layernorm_1 = nn.LayerNorm(d_model)
        self.layernorm_2 = nn.LayerNorm(d_model)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attention_output = self.diff_multi_head_attention(x, x, x, mask)
        out1 = self.layernorm_1(x + self.dropout(attention_output))

        ff_output = self.feed_forward(out1)
        out2 = self.layernorm_2(out1 + self.dropout(ff_output))

        return out2


In [7]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.diff_multi_head_attention_1 = DiffMultiHeadAttention(d_model, num_heads)
        self.diff_multi_head_attention_2 = DiffMultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForward(d_model, d_ff)

        self.layernorm_1 = nn.LayerNorm(d_model)
        self.layernorm_2 = nn.LayerNorm(d_model)
        self.layernorm_3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        attention_output_1 = self.diff_multi_head_attention_1(x, x, x, look_ahead_mask)
        out1 = self.layernorm_1(x + self.dropout(attention_output_1))

        attention_output_2 = self.diff_multi_head_attention_2(out1, enc_output, enc_output, padding_mask)
        out2 = self.layernorm_2(out1 + self.dropout(attention_output_2))

        ff_output = self.feed_forward(out2)
        out3 = self.layernorm_3(out2 + self.dropout(ff_output))

        return out3


In [8]:
class Transformer(nn.Module):
    def __init__(self, num_layers, d_model, num_heads, d_ff, input_vocab_size, target_vocab_size, max_seq_len, dropout):
        super(Transformer, self).__init__()
        
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_len)
        
        self.encoder_layers = nn.ModuleList(EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers))
        self.decoder_layers = nn.ModuleList(DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers))
        self.final_layer = nn.Linear(d_model, target_vocab_size)

        self.dropout = nn.Dropout(dropout)

    def encode(self, src, src_mask):
        src = self.encoder_embedding(src)
        src = self.positional_encoding(src)
        
        for layer in self.encoder_layers:
            src = layer(src, src_mask)

        return src
    
    def decode(self, target, memory, target_mask, src_mask):
        tgt = self.decoder_embedding(target)
        tgt = self.positional_encoding(tgt)

        for layer in self.decoder_layers:
            tgt = layer(tgt, memory, target_mask, src_mask)

        return tgt
    
    def forward(self, src, target, src_mask, target_mask):
        enc_output = self.encode(src, src_mask)
        dec_output = self.decode(target, enc_output, target_mask, src_mask)

        final_output = self.final_layer(dec_output)

        return final_output


In [9]:
# usage
src_vocab_size = 5000
tgt_vocab_size = 5000
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_len = 100  # Make sure this matches or exceeds your maximum sequence length
dropout = 0.1

transformer = Transformer(num_layers, d_model, num_heads, d_ff, src_vocab_size, tgt_vocab_size, max_seq_len, dropout)


In [10]:
# sample input
batch_size = 64
seq_length = 50
src = torch.randint(1, src_vocab_size, (batch_size, seq_length))
tgt = torch.randint(1, tgt_vocab_size, (batch_size, seq_length)) 

# create masks 
src_mask = torch.ones(batch_size, 1, 1, seq_length)
tgt_mask = torch.tril(torch.ones(batch_size, 1, seq_length, seq_length))

# forward pass
output = transformer(src, tgt, src_mask, tgt_mask)
print(output.shape)  # Expected: torch.Size([64, 50, 5000])

# training loop 
epochs = 500

loss_fn = nn.CrossEntropyLoss(ignore_index=0) 
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

for epoch in range(epochs): 
    optimizer.zero_grad()
    output = transformer(src, tgt, src_mask, tgt_mask)
    loss = loss_fn(output.view(-1, tgt_vocab_size), tgt.view(-1))
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

print("Training completed!")


torch.Size([64, 50, 5000])
Epoch 0, Loss: 8.671402931213379
Epoch 10, Loss: 7.059451103210449
Epoch 20, Loss: 5.545629978179932
Epoch 30, Loss: 4.181428909301758
Epoch 40, Loss: 3.055572748184204
Epoch 50, Loss: 2.170642614364624
Epoch 60, Loss: 1.4853633642196655
Epoch 70, Loss: 0.9765584468841553
Epoch 80, Loss: 0.6180844306945801
Epoch 90, Loss: 0.3937240242958069
Epoch 100, Loss: 0.2623292803764343
Epoch 110, Loss: 0.18651872873306274
Epoch 120, Loss: 0.13920468091964722
Epoch 130, Loss: 0.10877159237861633
Epoch 140, Loss: 0.08754243701696396
Epoch 150, Loss: 0.0723409429192543
Epoch 160, Loss: 0.060705047100782394
Epoch 170, Loss: 0.05175051838159561
Epoch 180, Loss: 0.04458371177315712
Epoch 190, Loss: 0.03848520666360855
Epoch 200, Loss: 0.03357745334506035
Epoch 210, Loss: 0.02946525812149048
Epoch 220, Loss: 0.02588602527976036
Epoch 230, Loss: 0.022866351529955864
Epoch 240, Loss: 0.020365538075566292
Epoch 250, Loss: 0.01803489215672016
Epoch 260, Loss: 0.016027770936489105